# Постановка задачи

Задачей предусматриваются перевозки различным типом грузового транспорта от поставщиков к потребителям по прямым маршрутам. Обозначим множество(количество) поставщиков через Warhouses, множество(количество) потребителей – Consumers, множество(количество) различных типов транспорта через TransportType. 

Для каждого маршрута Warhouse-Consumer задана стоимость перевозки определенным типом транспорта:
 〖routeCost〗_ijk,i=1..Warhouses,j=1..Consumers,k=1..TransportType.
 
Для каждого поставщика задано количество транспорта, доступное для перевозок от этого поставщика:
〖WareTransportQuantity〗_ik,i=1..Warhouses,k=1..TransportType, 
и объем товара на балансе поставщика, доступный к перевозке:
〖WareVolume〗_i,i=1..Warhouses.

Для каждого потребителя задан спрос (количество товара, которое необходимо привезти потребителю):
〖Consumers〗_j,j=1..Consumers .

Для каждого типа транспорта задана его грузоподъёмность:
〖TransportType〗_k,k=1..TransportType.

Цель: исходя из спроса на потребителях и предложения на поставщиках, построить план перевозок от поставщиков к потребителям с учетом доступного транспорта при минимизации издержек на транспортировку.

Задачу необходимо решить в Excel 2010 или на любом из следующих языков программирования: C++, C#, Java, Matlab, OPL.

# Формулировка модели

Sets:

$W$ – warehouses

$C$ – customers

$TT$ – transport type

Parameters:

$S_{w}$ – supply at warehouse $w$

$D_{c}$ – demand by customer $c$

$U_{w,c,tt}$ – unit transport cost from $w$ to $c$ by transport type $tt$

$LC_{tt}$ – load capacity by transport type $tt$

$AT_{w,tt}$ – available transport $tt$ at warehouse $w$  

Variables:

$x_{w,c,tt}$ – transport value from $w$ to $c$ by transport type $tt$

$z$ – total transport cost

Minimize: 

$z = \sum_{w}\sum_{c}\sum_{tt} U_{w, c, tt} · x_{w, c, tt}$

subject to:

$\sum_{c}\sum_{tt} x_{w,c,tt} ≤ S_{w}$

$\sum_{w}\sum_{tt} x_{w,c,tt} = D_{c}$

$\sum_{c} x_{w, c, tt} ≤ LC_{tt} · AT_{w, tt}$

$x_{w, c, tt} ≥ 0  ∀(w, c, tt)$ 


# Реализация

Импортируем библиотеки

In [1]:
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

Берем данные из внешего источника

In [2]:
path = (r"02 Data.xls")
df_demand = pd.read_excel(path, sheet_name='Demand')
df_supply = pd.read_excel(path, sheet_name='Supply')
df_costs = pd.read_excel(path, sheet_name='Costs')
df_carTypes = pd.read_excel(path, sheet_name='TType')
df_dislocation = pd.read_excel(path, sheet_name='Dislocation')

Начинаем писать модель

In [3]:
model = pyo.ConcreteModel('Transport Problem with car types')

Множества

In [4]:
model.customers_set = list(df_demand.set_index(['cons_name']).index)
model.warehouses_set = list(df_supply.set_index(['ware_name']).index)
model.carTypes_set = list(df_carTypes.set_index(['transport_type']).index)

model.costs_set = list(df_costs.set_index(['ware_name', 'cons_name', 'transport_type']).index)
model.dislocation_set = list(df_dislocation.set_index(['ware_name', 'transport_type']).index)

In [5]:
model.s_customers = pyo.Set(initialize = model.customers_set)
model.s_warehouses = pyo.Set(initialize = model.warehouses_set)
model.s_carTypes = pyo.Set(initialize = model.carTypes_set)

Множества с числом индексов больше одного

In [6]:
model.s_costs = pyo.Set(initialize = model.costs_set)
model.s_dislocation = pyo.Set(initialize = model.dislocation_set)

Параметры

In [7]:
model.p_demand = pyo.Param(model.s_customers, initialize = df_demand.set_index(['cons_name'])['cons_demand, кг товара'].to_dict())
model.p_supply = pyo.Param(model.s_warehouses, initialize = df_supply.set_index(['ware_name'])['ware_volume, кг товара'].to_dict())
model.p_loadCapacity = pyo.Param(model.s_carTypes, initialize = df_carTypes.set_index(['transport_type'])['transport_ability, кг товара'].to_dict())
model.p_cost = pyo.Param(model.s_costs, initialize = df_costs.set_index(['ware_name', 'cons_name', 'transport_type'])['transport_cost (стоимость в у.е.)'].to_dict())
model.p_availableCars = pyo.Param(model.s_dislocation, initialize = df_dislocation.set_index(['ware_name', 'transport_type'])['transport_quantity (количество доступного транспорта в ед.)'].to_dict())

Переменные

In [8]:
model.v_x = pyo.Var(model.s_costs, domain=pyo.NonNegativeReals)

Целевая функция

In [9]:
def OF_rule(model):
    return sum(model.v_x[(w, c, tt)] * model.p_cost[(w, c, tt)] for (w, c, tt) in model.s_costs)
model.v_OF = pyo.Objective(rule=OF_rule, sense=pyo.minimize)

Ограничения

"Index '('Yaroslavl', 'Vladimir', '20 tonn')' is not valid for indexed component 'v_x'" так как нет данных по 20ти тоннику - т.е. нужно ограничить что индексы должны входить в model.p_availableCars

In [10]:
index_domain_w = df_costs.assign(index_domain=df_costs[['cons_name', 'transport_type']].to_records(index=False).tolist()).groupby(['ware_name']).agg(set).index_domain.to_dict()

In [11]:
def Supply_rule(model, w):
    return sum(model.v_x[(w, c, tt)] for c, tt in index_domain_w.get(w, [])) <= model.p_supply[w]
model.c_Supply = pyo.Constraint(model.s_warehouses, rule=Supply_rule)

In [12]:
index_domain_c = df_costs.assign(index_domain=df_costs[['ware_name', 'transport_type']].to_records(index=False).tolist()).groupby(['cons_name']).agg(set).index_domain.to_dict()

In [13]:
def Demand_rule(model, c):
    return sum(model.v_x[(w, c, tt)] for w, tt in index_domain_c.get(c, [])) == model.p_demand[c]
model.c_Demand = pyo.Constraint(model.s_customers, rule=Demand_rule)

In [14]:
index_domain_tt = df_costs.assign(index_domain=df_costs[['cons_name']]).groupby(['ware_name', 'transport_type']).agg(set).index_domain.to_dict()

In [15]:
def Transport_rule(model, w, tt):
    return sum(model.v_x[(w, c, tt)] for c in index_domain_tt.get((w, tt), [])) <= model.p_loadCapacity[tt] * model.p_availableCars[w, tt]
model.c_Transport = pyo.Constraint(model.s_warehouses, model.s_carTypes, rule=Transport_rule)

Выбираем солвер

In [16]:
solver = pyo.SolverFactory('gurobi')

Создаем суффикс для анализа чувствительсности

In [17]:
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Решаем модель

In [18]:
result = solver.solve(model)

Смотрим на целевую

In [19]:
model.v_OF()

11830000.0

Выводим результаты

In [20]:
print(result)


Problem: 
- Name: x1
  Lower bound: 11830000.0
  Upper bound: 11830000.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 23
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 23
  Number of nonzeros: 69
  Sense: minimize
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Wall time: 0.11700010299682617
  Error rc: 0
  Time: 0.627129316329956
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



Проверяем, как сгенерировалась модель

In [21]:
model.display()

Model Transport Problem with car types

  Variables:
    v_x : Size=23, Index=s_costs
        Key                                  : Lower : Value    : Upper : Fixed : Stale : Domain
             ('Moscow', 'Bransk', '10 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
             ('Moscow', 'Bransk', '20 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
              ('Moscow', 'Bransk', '3 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
              ('Moscow', 'Bransk', '5 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
               ('Moscow', 'Orel', '10 tonn') :     0 : 100000.0 :  None : False : False : NonNegativeReals
               ('Moscow', 'Orel', '20 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
                ('Moscow', 'Orel', '3 tonn') :     0 :      0.0 :  None : False : False : NonNegativeReals
                ('Moscow', 'Orel', '5 tonn') :     0 :      0.0 :  N

Записываем lp-файл с моделью

In [22]:
model.write('02 model.lp', io_options={'symbolic_solver_labels': True})

('02 model.lp', 2277552318160)

## Анализ на чувствительность 

In [23]:
print("Duals")
for c in model.component_objects(pyo.Constraint, active=True):
    print("   Constraint", c)
    for index in c:
        print("      ", index, model.dual[c[index]])

Duals
   Constraint c_Supply
       Moscow 0.0
       Yaroslavl 0.0
   Constraint c_Demand
       Orel 27.0
       Bransk 30.0
       Vladimir 29.0
   Constraint c_Transport
       ('Moscow', '3 tonn') -19.0
       ('Moscow', '5 tonn') -16.0
       ('Moscow', '10 tonn') -11.0
       ('Moscow', '20 tonn') -5.0
       ('Yaroslavl', '3 tonn') -13.0
       ('Yaroslavl', '5 tonn') -10.0
       ('Yaroslavl', '10 tonn') -4.0
       ('Yaroslavl', '20 tonn') 0.0


Изменение спроса на единицу увеличит целевую на величину значения двойственной оценки для c_Demand